```sh
git clone https://github.com/finos/legend-delta.git
cd python
python3 setup.py bdist_wheel
```

## Legend model
Legend project can be loaded from classpath or directory as follows

In [0]:
from legend.delta import LegendClasspathLoader
legend = LegendClasspathLoader().loadResources()

## Legend schema
We can create the spark schema for any Legend entity of type `Class`. 
This process will recursively loop through each of its underlying fields, enums and possibly nested properties and supertypes.

In [0]:
schema = legend.get_schema("databricks::entity::employee")

In [0]:
import pandas as pd
pd.DataFrame(
  [[f.name, str(f.dataType), f.nullable, f.metadata['comment']] for f in schema.fields], 
  columns=['field', 'type', 'optional', 'description']
)

Out[9]:

,field,type,optional,description
0,firstName,StringType,False,Person first name
1,lastName,StringType,False,Person last name
2,birthDate,DateType,False,Person birth date
3,gender,StringType,True,Person gender
4,id,IntegerType,False,Unique identifier of a databricks employee
5,sme,StringType,True,Programming skill that person truly masters
6,joinedDate,DateType,False,When did that person join Databricks
7,highFives,IntegerType,True,How many high fives did that person get


## Legend expectations
Given the `multiplicity` properties, we can 
detect if a field is optional or not or list has the right number of elements. Given an `enumeration`, 
we check for value consistency. These will be considered **technical expectations** and converted into SQL constraints.

In [0]:
expectations = legend.get_expectations("databricks::entity::employee")

In [0]:
import pandas as pd
pd.DataFrame(
  [[e, expectations[e]] for e in expectations], 
  columns=['expectation', 'constraint']
)

Out[10]:

,expectation,constraint
0,[birthDate] is mandatory,birthDate IS NOT NULL
1,[sme] not allowed value,"sme IS NULL OR sme IN ('Scala', 'Python', 'C',..."
2,[id] is mandatory,id IS NOT NULL
3,[joinedDate] is mandatory,joinedDate IS NOT NULL
4,[firstName] is mandatory,firstName IS NOT NULL
5,[lastName] is mandatory,lastName IS NOT NULL


In addition to the rules derived from the schema itself, we also support the conversion of **business expectations**
from the PURE language to SQL expressions. We generate a legend
execution plan against a Databricks runtime, hence operating against relational legend `mapping` rather
than pure entities of type `class`.

In [0]:
expectations = legend.get_expectations("databricks::mapping::employee_delta")

In [0]:
import pandas as pd
pd.DataFrame(
  [[e, expectations[e]] for e in expectations], 
  columns=['expectation', 'constraint']
)

Out[12]:

,expectation,constraint
0,[birthDate] is mandatory,birth_date IS NOT NULL
1,[sme] not allowed value,"(sme IS NULL OR sme IN ('Scala', 'Python', 'C'..."
2,[id] is mandatory,id IS NOT NULL
3,[joinedDate] is mandatory,joined_date IS NOT NULL
4,[firstName] is mandatory,first_name IS NOT NULL
5,[high five] should be positive,(high_fives IS NOT NULL AND high_fives > 0)
6,[age] should be > 21,year(joined_date) - year(birth_date) > 21
7,[lastName] is mandatory,last_name IS NOT NULL


## Legend transformations
In addition to business expectations, we transform raw entities into their
desired states and target tables. Note that relational transformations on legend only support direct mapping 
(no PURE operations or derived properties) and therefore easily enforced through `.withColumnRenamed` syntax.

In [0]:
transformations = legend.get_transformations("databricks::mapping::employee_delta")

In [0]:
import pandas as pd
pd.DataFrame(
  [[e, transformations[e]] for e in transformations], 
  columns=['from_column', 'to_column']
)

Out[14]:

,from_column,to_column
0,highFives,high_fives
1,joinedDate,joined_date
2,lastName,last_name
3,firstName,first_name
4,birthDate,birth_date
5,id,id
6,sme,sme
7,gender,gender


## Legend tables
In order to query our validated entity from legend interface, we can easily create the target state table. This table contains a placeholder for our invalidated constraints (below field `legend`).

In [0]:
table_path = '/FileStore/legend/employee'
table_name = legend.create_table("databricks::mapping::employee_delta", table_path, 'legend')

In [0]:
display(sql("DESCRIBE EXTENDED {}".format(table_name)))

col_name,data_type,comment
first_name,string,Person first name
last_name,string,Person last name
birth_date,date,Person birth date
gender,string,Person gender
id,int,Unique identifier of a databricks employee
sme,string,Programming skill that person truly masters
joined_date,date,When did that person join Databricks
high_fives,int,How many high fives did that person get
legend,array,LEGEND VIOLATION FIELD
,,
